# Lung Histopathology Classification: ACA / N / SCC
## Multi-CNN + Channel Attention + GA + KNN/SVM/RF + Fusion

This notebook implements a comprehensive lung histopathology classification system that combines:
- Multiple CNN backbones (DenseNet121, ResNet50, VGG16)
- Channel attention mechanism (SE blocks)
- Genetic Algorithm for feature selection
- Ensemble of classical ML classifiers (KNN, SVM, Random Forest)
- Majority voting fusion

In [50]:
!pip install --upgrade --force-reinstall numpy==1.25.2 tensorflow==2.15.0 keras==2.15.0


  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.7 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached protobuf-4.25.8-cp37-abi3-manylinux20

In [ ]:
# Import required libraries
import os, random, json
import numpy as np
import pandas as pd
import keras
import tensorflow as tf

from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.applications import DenseNet121, ResNet50, EfficientNetB0, InceptionV3
from tensorflow.keras.applications.densenet import preprocess_input as pre_densenet
from tensorflow.keras.applications.resnet import preprocess_input as pre_resnet
from tensorflow.keras.applications.efficientnet import preprocess_input as pre_efficientnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as pre_inception
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, GlobalMaxPooling2D,
                                     Concatenate, Dense, Reshape, Multiply, Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import mode

# from deap import base, creator, tools  # GA removed, not needed

print("All libraries imported successfully!")

2025-08-22 09:13:25.094098: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-22 09:13:25.406044: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-22 09:13:25.406265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-22 09:13:25.455157: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-22 09:13:25.573868: I tensorflow/core/platform/cpu_feature_guar

All libraries imported successfully!


In [2]:
# Configuration and Data Setup
DATA_DIR   = "/teamspace/studios/this_studio/lung_cancer/dataset/lung_image_sets"  # << set this
IMG_SIZE   = (224, 224)
BATCH_SIZE = 24
SEED       = 42

# Set random seeds for reproducibility
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(f"Configuration set:")
print(f"Data Directory: {DATA_DIR}")
print(f"Image Size: {IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Random Seed: {SEED}")

Configuration set:
Data Directory: /teamspace/studios/this_studio/lung_cancer/dataset/lung_image_sets
Image Size: (224, 224)
Batch Size: 24
Random Seed: 42


In [3]:
# === Global Model Hyperparameters ===
# Number of attention heads for multi-head channel attention
NUM_ATTENTION_HEADS = 8

In [4]:
# Data Generators Setup
# Only lung classes will be present in this directory if you set DATA_DIR as above:
# expected subfolders: lung_aca / lung_n / lung_scc

train_datagen = ImageDataGenerator(
    validation_split=0.20,
    rotation_range=20,
    horizontal_flip=True,
    # IMPORTANT: no rescale here, since we feed raw to model-specific preprocessors
)

def make_gen(subset):
    return train_datagen.flow_from_directory(
        DATA_DIR,
        target_size=IMG_SIZE,
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        subset=subset,
        seed=SEED,
        shuffle=True
    )

train_gen = make_gen('training')
val_gen   = make_gen('validation')
num_classes = train_gen.num_classes
class_indices = train_gen.class_indices
id2label = {v:k for k,v in class_indices.items()}

print("Classes:", class_indices)
print(f"Number of classes: {num_classes}")
print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")

Found 12000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.
Classes: {'lung_aca': 0, 'lung_n': 1, 'lung_scc': 2}
Number of classes: 3
Training samples: 12000
Validation samples: 3000


In [5]:
# Channel Attention (Multi-Headed) Implementation

import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Reshape, Permute, Concatenate

class MultiHeadChannelAttention(Layer):
    def __init__(self, num_heads=4, reduction=16, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.reduction = reduction

    def build(self, input_shape):
        self.channel = input_shape[-1]
        self.dense1 = [Dense(self.channel // self.reduction, activation='relu') for _ in range(self.num_heads)]
        self.dense2 = [Dense(self.channel) for _ in range(self.num_heads)]
        super().build(input_shape)

    def call(self, x):
        # Global pooling
        gap = tf.reduce_mean(x, axis=[1,2])  # shape: (batch, channels)
        gmp = tf.reduce_max(x, axis=[1,2])   # shape: (batch, channels)
        heads = []
        for i in range(self.num_heads):
            d1_gap = self.dense1[i](gap)
            d1_gmp = self.dense1[i](gmp)
            d2_gap = self.dense2[i](d1_gap)
            d2_gmp = self.dense2[i](d1_gmp)
            scale = tf.nn.sigmoid(d2_gap + d2_gmp)
            scale = Reshape((1,1,self.channel))(scale)
            heads.append(x * scale)
        # Concatenate heads along channel axis
        out = Concatenate(axis=-1)(heads)
        return out

# Usage in your lane function:
# from multi_head_attention import MultiHeadChannelAttention
# x = MultiHeadChannelAttention(num_heads=4, reduction=16)(x)

def multi_head_attention_block(x, reduction=16, name=None):
    """Multi-Headed Channel Attention block for CNN feature maps"""
    attn = MultiHeadChannelAttention(num_heads=NUM_ATTENTION_HEADS, reduction=reduction, name=name)(x)
    return attn

print("Multi-head attention block function defined successfully!")


Multi-head attention block function defined successfully!


In [ ]:
# Preprocessing Lanes (one per backbone)
def lane(tensor, backbone="resnet", reduction=16):
    """Create a processing lane for each CNN backbone with multi-head channel attention"""
    if backbone == "resnet":
        x = Lambda(pre_resnet, name="pre_resnet")(tensor)
        x = ResNet50(include_top=False, weights='imagenet')(x)
    elif backbone == "densenet":
        x = Lambda(pre_densenet, name="pre_densenet")(tensor)
        x = DenseNet121(include_top=False, weights='imagenet')(x)
    elif backbone == "efficientnet":
        x = Lambda(pre_efficientnet, name="pre_efficientnet")(tensor)
        x = EfficientNetB0(include_top=False, weights='imagenet')(x)
    elif backbone == "inception":
        x = Lambda(pre_inception, name="pre_inception")(tensor)
        x = InceptionV3(include_top=False, weights='imagenet')(x)
    else:
        raise ValueError(f'Unknown backbone: {backbone}')
    # Add multi-head channel attention
    x = multi_head_attention_block(x, reduction=reduction, name=f"mhca_{backbone}")
    # Global Average Pooling to convert feature maps → vector
    x = GlobalAveragePooling2D(name=f"gap_{backbone}")(x)
    return x

print("Lane function updated for multi-head attention!")


Lane function updated for multi-head attention!


In [ ]:
# Build Feature Extractor Model
print("Building multi-backbone feature concatenator with multi-head attention...")

# Define input tensor with image size (224x224x3 RGB)
inp = Input(shape=(224,224,3))

# Extract features from DenseNet lane (multi-head attention)
feat_d = lane(inp, "densenet", reduction=16)
# Extract features from ResNet lane (multi-head attention)
feat_r = lane(inp, "resnet", reduction=16)
# Extract features from EfficientNetB0 lane (multi-head attention)
feat_e = lane(inp, "efficientnet", reduction=16)
# Extract features from InceptionV3 lane (multi-head attention)
feat_i = lane(inp, "inception", reduction=16)

# Concatenate features from all four backbones
concat_feat = Concatenate(name="concat_feats")([feat_d, feat_r, feat_e, feat_i])

# Create feature extractor model (input → concatenated features)
feature_model = Model(inp, concat_feat)

# Get final concatenated feature dimension
feature_dim = feature_model.output_shape[-1]

print(f"Feature extractor built successfully!")
print(f"Feature dimension: {feature_dim}")

# Show model summary (layers, parameters, shapes)
feature_model.summary()


Building multi-backbone feature concatenator with multi-head attention...


2025-08-22 09:13:52.407529: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20763 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:04.0, compute capability: 8.9


Feature extractor built successfully!
Feature dimension: 28672
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 pre_densenet (Lambda)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 pre_resnet (Lambda)         (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 pre_vgg (Lambda)            (None, 224, 224, 3)          0         ['input_1[0][0]']             
                               

In [8]:
# Extract Deep Features
def extract_features(generator):
    """Extract features from a data generator using the feature model"""
    X, y = [], []
    steps = len(generator)
    for i in range(steps):
        imgs, labels = generator.next()
        feats = feature_model.predict(imgs, verbose=0)
        X.append(feats)
        y.append(labels)
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{steps} batches")
    return np.vstack(X), np.vstack(y)

print("Feature extraction function defined!")

Feature extraction function defined!


In [ ]:
# Feature Selection: mRMR + Adaptive Grey Wolf Optimization (AGWO)
# Remove GA-based feature selection. Use mRMR for initial ranking, then AGWO for final selection.
# You may need to install pymrmr and a Grey Wolf Optimizer package, or implement AGWO manually.
# Example below assumes features (X) and labels (y) are available after extraction.

## 1. mRMR Feature Ranking
try:
    import pymrmr
except ImportError:
    !pip install pymrmr
    import pymrmr

# Convert features and labels to DataFrame for pymrmr
import pandas as pd
def mrmr_feature_selection(X, y, n_features=100):
    df = pd.DataFrame(X)
    df['target'] = np.argmax(y, axis=1)
    selected = pymrmr.mRMR(df, 'MIQ', n_features)
    return selected

# Example usage:
## selected_features = mrmr_feature_selection(X_train, y_train, n_features=100)

## 2. Adaptive Grey Wolf Optimization (AGWO)
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

def agwo_feature_selection(X, y, ranked_features, n_wolves=10, n_iter=10, subset_size=30):
    # Only use the top ranked features from mRMR
    feature_indices = [int(f) for f in ranked_features]
    X_ranked = X[:, feature_indices]
    n_features = len(feature_indices)
    # Initialize wolves (random binary vectors)
    wolves = np.random.randint(0, 2, size=(n_wolves, n_features))
    # Ensure at least subset_size features are selected
    for w in wolves:
        idx = np.where(w == 1)[0]
        if len(idx) < subset_size:
            w[:subset_size] = 1
    alpha, beta, delta = None, None, None
    alpha_score, beta_score, delta_score = -np.inf, -np.inf, -np.inf

    for it in range(n_iter):
        for i, wolf in enumerate(wolves):
            idx = np.where(wolf == 1)[0]
            if len(idx) == 0: continue
            X_sel = X_ranked[:, idx]
            # Use KNN accuracy as fitness
            score = cross_val_score(KNeighborsClassifier(n_neighbors=5), X_sel, np.argmax(y, axis=1), cv=3).mean()
            if score > alpha_score:
                delta, delta_score = beta, beta_score
                beta, beta_score = alpha, alpha_score
                alpha, alpha_score = wolf.copy(), score
            elif score > beta_score:
                delta, delta_score = beta, beta_score
                beta, beta_score = wolf.copy(), score
            elif score > delta_score:
                delta, delta_score = wolf.copy(), score
        # Update wolves positions (simplified AGWO)
        for i, wolf in enumerate(wolves):
            for j in range(n_features):
                r = np.random.rand()
                if r < 0.33:
                    wolf[j] = alpha[j]
                elif r < 0.66:
                    wolf[j] = beta[j]
                else:
                    wolf[j] = delta[j]
            # Random mutation
            if np.random.rand() < 0.1:
                flip = np.random.randint(0, n_features)
                wolf[flip] = 1 - wolf[flip]
    # Return indices of selected features from the best wolf (alpha)
    selected_idx = np.where(alpha == 1)[0]
    selected_features = [feature_indices[i] for i in selected_idx]
    return selected_features

print("Feature selection pipeline (mRMR + AGWO) implemented.")

Extracting training features …


2025-08-22 09:14:08.944367: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8900


Processed 10/500 batches
Processed 20/500 batches
Processed 30/500 batches
Processed 40/500 batches
Processed 50/500 batches
Processed 60/500 batches
Processed 70/500 batches
Processed 80/500 batches
Processed 90/500 batches
Processed 100/500 batches
Processed 110/500 batches
Processed 120/500 batches
Processed 130/500 batches
Processed 140/500 batches
Processed 150/500 batches
Processed 160/500 batches
Processed 170/500 batches
Processed 180/500 batches
Processed 190/500 batches
Processed 200/500 batches
Processed 210/500 batches
Processed 220/500 batches
Processed 230/500 batches
Processed 240/500 batches
Processed 250/500 batches
Processed 260/500 batches
Processed 270/500 batches
Processed 280/500 batches
Processed 290/500 batches
Processed 300/500 batches
Processed 310/500 batches
Processed 320/500 batches
Processed 330/500 batches
Processed 340/500 batches
Processed 350/500 batches
Processed 360/500 batches
Processed 370/500 batches
Processed 380/500 batches
Processed 390/500 bat

In [10]:
# Extract Validation Features
print("Extracting validation features …")
X_va, Y_va_ohe = extract_features(val_gen)
print(f"Validation features shape: {X_va.shape}")
print(f"Validation labels shape: {Y_va_ohe.shape}")

Extracting validation features …


Processed 10/125 batches
Processed 20/125 batches
Processed 30/125 batches
Processed 40/125 batches
Processed 50/125 batches
Processed 60/125 batches
Processed 70/125 batches
Processed 80/125 batches
Processed 90/125 batches
Processed 100/125 batches
Processed 110/125 batches
Processed 120/125 batches
Validation features shape: (3000, 28672)
Validation labels shape: (3000, 3)


In [11]:
# Combine Features and Convert Labels
X_full = np.vstack([X_tr, X_va])
y_full = np.argmax(np.vstack([Y_tr_ohe, Y_va_ohe]), axis=1)

print(f"Total features shape: {X_full.shape}")
print(f"Total labels shape: {y_full.shape}")
print(f"Classes present: {np.unique(y_full)}")
print(f"Class distribution: {np.bincount(y_full)}")

Total features shape: (15000, 28672)
Total labels shape: (15000,)
Classes present: [0 1 2]
Class distribution: [5000 5000 5000]


In [12]:
# GA-based Feature Selection Setup (DEAP)
POP_SIZE = 40
N_GEN    = 10        # start smaller; increase later
CX_PROB  = 0.8
MUT_PROB = 0.1
INDPB    = 0.05

n_features = X_full.shape[1]

print(f"GA Parameters:")
print(f"Population Size: {POP_SIZE}")
print(f"Generations: {N_GEN}")
print(f"Crossover Probability: {CX_PROB}")
print(f"Mutation Probability: {MUT_PROB}")
print(f"Total Features: {n_features}")

GA Parameters:
Population Size: 40
Generations: 10
Crossover Probability: 0.8
Mutation Probability: 0.1
Total Features: 28672


In [13]:
# Define GA Components
# Safe (re)definition guards for repeated runs
if "FitnessMax" not in creator.__dict__:
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if "Individual" not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_fitness(individual):
    """Evaluate fitness of an individual (feature subset)"""
    idx = [i for i, b in enumerate(individual) if b == 1]
    if len(idx) < 2:
        return (0.0,)
    Xs = X_full[:, idx]
    knn = KNeighborsClassifier(n_neighbors=5)
    scores = cross_val_score(knn, Xs, y_full, cv=3, scoring='accuracy')
    # Small L0 penalty to prefer compact subsets
    fitness = scores.mean() - 0.1 * (len(idx) / n_features)
    return (float(fitness),)

toolbox.register("evaluate", eval_fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=INDPB)
toolbox.register("select", tools.selTournament, tournsize=3)

print("GA components defined successfully!")

GA components defined successfully!


In [14]:
# Initialize GA Population
pop = toolbox.population(n=POP_SIZE)
print(f"GA initialized: pop={POP_SIZE}, feats={n_features}")
print("Starting genetic algorithm evolution...")

GA initialized: pop=40, feats=28672
Starting genetic algorithm evolution...


In [15]:
# Run GA Evolution
for gen in range(N_GEN):
    print(f"\nGeneration {gen+1}/{N_GEN}")
    
    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    # Crossover
    for c1, c2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CX_PROB:
            toolbox.mate(c1, c2)
            if "fitness" in c1.__dict__: del c1.fitness.values
            if "fitness" in c2.__dict__: del c2.fitness.values

    # Mutation
    for ind in offspring:
        if random.random() < MUT_PROB:
            toolbox.mutate(ind)
            if "fitness" in ind.__dict__: del ind.fitness.values

    # Evaluation
    invalid = [ind for ind in offspring if not ind.fitness.valid]
    print(f"  Evaluating {len(invalid)} individuals...")
    fits = list(map(toolbox.evaluate, invalid))
    for ind, fit in zip(invalid, fits):
        ind.fitness.values = fit

    pop[:] = offspring
    gen_fits = [ind.fitness.values[0] for ind in pop]
    print(f"  Max fitness: {np.max(gen_fits):.4f}")
    print(f"  Avg fitness: {np.mean(gen_fits):.4f}")

print("\nGA evolution completed!")


Generation 1/10


  Evaluating 40 individuals...
  Max fitness: 0.9265
  Avg fitness: 0.9224

Generation 2/10
  Evaluating 29 individuals...
  Max fitness: 0.9265
  Avg fitness: 0.9244

Generation 3/10
  Evaluating 33 individuals...
  Max fitness: 0.9271
  Avg fitness: 0.9258

Generation 4/10
  Evaluating 33 individuals...
  Max fitness: 0.9274
  Avg fitness: 0.9265

Generation 5/10
  Evaluating 36 individuals...
  Max fitness: 0.9280
  Avg fitness: 0.9268

Generation 6/10
  Evaluating 25 individuals...
  Max fitness: 0.9285
  Avg fitness: 0.9272

Generation 7/10
  Evaluating 32 individuals...
  Max fitness: 0.9289
  Avg fitness: 0.9276

Generation 8/10
  Evaluating 30 individuals...
  Max fitness: 0.9292
  Avg fitness: 0.9281

Generation 9/10
  Evaluating 31 individuals...
  Max fitness: 0.9298
  Avg fitness: 0.9283

Generation 10/10
  Evaluating 36 individuals...
  Max fitness: 0.9304
  Avg fitness: 0.9288

GA evolution completed!


In [16]:
# Select Best Features
best = tools.selBest(pop, 1)[0]
sel_idx = np.array([i for i, b in enumerate(best) if b == 1], dtype=int)

print(f"Selected {len(sel_idx)} / {n_features} features")
print(f"Feature selection ratio: {len(sel_idx)/n_features:.3f}")
print(f"Best fitness: {best.fitness.values[0]:.4f}")

Selected 14207 / 28672 features
Feature selection ratio: 0.496
Best fitness: 0.9304


In [17]:
# Prepare Selected Features for Training
X_sel = X_full[:, sel_idx]
X_train, X_test, y_train, y_test = train_test_split(
    X_sel, y_full, test_size=0.20, random_state=SEED, stratify=y_full
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Training class distribution: {np.bincount(y_train)}")
print(f"Test class distribution: {np.bincount(y_test)}")

Training set shape: (12000, 14207)
Test set shape: (3000, 14207)
Training class distribution: [4000 4000 4000]
Test class distribution: [1000 1000 1000]


In [ ]:
# Initialize Classifiers
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
svm = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale', random_state=SEED)
rf  = RandomForestClassifier(n_estimators=300, random_state=SEED, n_jobs=-1)
xgb = XGBClassifier(n_estimators=200, random_state=SEED, use_label_encoder=False, eval_metric='mlogloss')
lr  = LogisticRegression(max_iter=1000, random_state=SEED, n_jobs=-1)

print("Classifiers initialized:")
print(f"  KNN: k=5, weights='distance'")
print(f"  SVM: RBF kernel, C=1.0, gamma='scale'")
print(f"  Random Forest: 300 trees")
print(f"  XGBoost: 200 estimators")
print(f"  Logistic Regression: max_iter=1000")

Classifiers initialized:
  KNN: k=5, weights='distance'
  SVM: RBF kernel, C=1.0, gamma='scale'
  Random Forest: 300 trees


In [ ]:
# Train Classifiers
print("Training classifiers …")

print("  Training KNN...")
knn.fit(X_train, y_train)

print("  Training SVM...")
svm.fit(X_train, y_train)

print("  Training Random Forest...")
rf.fit(X_train, y_train)

print("  Training XGBoost...")
xgb.fit(X_train, y_train)

print("  Training Logistic Regression...")
lr.fit(X_train, y_train)

print("All classifiers trained successfully!")

Training classifiers …
  Training KNN...
  Training SVM...


  Training Random Forest...
All classifiers trained successfully!


In [ ]:
# Make Predictions
print("Making predictions...")

knn_pred = knn.predict(X_test)
svm_pred = svm.predict(X_test)
rf_pred  = rf.predict(X_test)
xgb_pred = xgb.predict(X_test)
lr_pred  = lr.predict(X_test)

# Probabilistic predictions (for ensemble if needed)
knn_proba = knn.predict_proba(X_test) if hasattr(knn, 'predict_proba') else None
svm_proba = svm.predict_proba(X_test) if hasattr(svm, 'predict_proba') else None
rf_proba  = rf.predict_proba(X_test) if hasattr(rf, 'predict_proba') else None
xgb_proba = xgb.predict_proba(X_test) if hasattr(xgb, 'predict_proba') else None
lr_proba  = lr.predict_proba(X_test) if hasattr(lr, 'predict_proba') else None

print("Predictions completed!")

Making predictions...


Predictions completed!


In [ ]:
# Individual Classifier Results
print("Individual Classifier Accuracies:")
knn_acc = accuracy_score(y_test, knn_pred)
svm_acc = accuracy_score(y_test, svm_pred)
rf_acc = accuracy_score(y_test, rf_pred)
xgb_acc = accuracy_score(y_test, xgb_pred)
lr_acc = accuracy_score(y_test, lr_pred)

print(f"  KNN: {knn_acc:.4f}")
print(f"  SVM: {svm_acc:.4f}")
print(f"  RF : {rf_acc:.4f}")
print(f"  XGB: {xgb_acc:.4f}")
print(f"  LR : {lr_acc:.4f}")

# Display individual classification reports
target_names = [id2label[i] for i in range(num_classes)]

print("\n=== KNN Classification Report ===")
print(classification_report(y_test, knn_pred, target_names=target_names))

print("\n=== SVM Classification Report ===")
print(classification_report(y_test, svm_pred, target_names=target_names))

print("\n=== Random Forest Classification Report ===")
print(classification_report(y_test, rf_pred, target_names=target_names))

print("\n=== XGBoost Classification Report ===")
print(classification_report(y_test, xgb_pred, target_names=target_names))

print("\n=== Logistic Regression Classification Report ===")
print(classification_report(y_test, lr_pred, target_names=target_names))

Individual Classifier Accuracies:
  KNN: 0.9843
  SVM: 0.9817
  RF : 0.9753

=== KNN Classification Report ===
              precision    recall  f1-score   support

    lung_aca       0.99      0.96      0.98      1000
      lung_n       1.00      1.00      1.00      1000
    lung_scc       0.96      0.99      0.98      1000

    accuracy                           0.98      3000
   macro avg       0.98      0.98      0.98      3000
weighted avg       0.98      0.98      0.98      3000


=== SVM Classification Report ===
              precision    recall  f1-score   support

    lung_aca       0.98      0.96      0.97      1000
      lung_n       1.00      1.00      1.00      1000
    lung_scc       0.96      0.98      0.97      1000

    accuracy                           0.98      3000
   macro avg       0.98      0.98      0.98      3000
weighted avg       0.98      0.98      0.98      3000


=== Random Forest Classification Report ===
              precision    recall  f1-score   s

In [ ]:
# Ensemble Fusion (Priority-Based Strategy)
# Priority: SVM > XGBoost > RF > KNN > LR
# If SVM and XGBoost agree, use that prediction. Else, use SVM. If not, use XGBoost. Else, fallback to majority vote.
def priority_ensemble(svm_pred, xgb_pred, rf_pred, knn_pred, lr_pred):
    preds = np.stack([knn_pred, svm_pred, rf_pred, xgb_pred, lr_pred], axis=0)
    final = []
    for i in range(svm_pred.shape[0]):
        if svm_pred[i] == xgb_pred[i]:
            final.append(svm_pred[i])
        elif svm_pred[i] == rf_pred[i]:
            final.append(svm_pred[i])
        elif xgb_pred[i] == rf_pred[i]:
            final.append(xgb_pred[i])
        else:
            # fallback to majority vote
            vals, counts = np.unique(preds[:, i], return_counts=True)
            final.append(vals[np.argmax(counts)])
    return np.array(final)

ens = priority_ensemble(svm_pred, xgb_pred, rf_pred, knn_pred, lr_pred)
ens_acc = accuracy_score(y_test, ens)

print(f"Ensemble Accuracy (Priority-Based): {ens_acc:.4f}")
print(f"\nImprovement over best individual: {ens_acc - max(knn_acc, svm_acc, rf_acc, xgb_acc, lr_acc):.4f}")

print("\n=== Ensemble Classification Report ===")
print(classification_report(y_test, ens, target_names=target_names))

Ensemble Accuracy (Majority Voting): 0.9870

Improvement over best individual: 0.0027

=== Ensemble Classification Report ===
              precision    recall  f1-score   support

    lung_aca       0.99      0.97      0.98      1000
      lung_n       1.00      1.00      1.00      1000
    lung_scc       0.97      0.99      0.98      1000

    accuracy                           0.99      3000
   macro avg       0.99      0.99      0.99      3000
weighted avg       0.99      0.99      0.99      3000



In [24]:
# Summary Results
print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)
print(f"Total samples processed: {len(y_full)}")
print(f"Features selected by GA: {len(sel_idx)} / {n_features} ({len(sel_idx)/n_features:.1%})")
print(f"Test set size: {len(y_test)}")
print("\nClassifier Accuracies:")
print(f"  KNN:              {knn_acc:.4f}")
print(f"  SVM:              {svm_acc:.4f}")
print(f"  Random Forest:    {rf_acc:.4f}")
print(f"  Ensemble (Fusion): {ens_acc:.4f} ← BEST")
print("\nClass Labels:")
for i, label in id2label.items():
    print(f"  {i}: {label}")
print("="*60)


FINAL RESULTS SUMMARY
Total samples processed: 15000
Features selected by GA: 14207 / 28672 (49.6%)
Test set size: 3000

Classifier Accuracies:
  KNN:              0.9843
  SVM:              0.9817
  Random Forest:    0.9753
  Ensemble (Fusion): 0.9870 ← BEST

Class Labels:
  0: lung_aca
  1: lung_n
  2: lung_scc
